<a href="https://colab.research.google.com/github/PedrolyraC/Campo-Minado/blob/main/GWO_for_svm%2C_rbf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import configparser
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import random
import sklearn.model_selection
import time
import warnings

from sklearn import svm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR

In [ ]:
def sanitized_GWO(X_train,
                  X_test,
                  y_train,
                  y_test,
                  search_agents_num,
                  T,
                  dim,
                  lb,
                  ub
                  ):
    alpha_position = [0, 0]
    beta_position = [0, 0]
    delta_position = [0, 0]

    alpha_score = float('inf')
    beta_score = float('inf')
    delta_score = float('inf')

    positions = np.dot(np.rand(search_agents_num,dim),(ub - lb)) + lb
    convergence_curve = np.zeros((1,T))

    iterations = []
    accuracy = []
    t = 0

    while t < T:
        for i in range(0, positions.shape[0]):
            for j in range(0, positions.shape[1]):
                ub_flag = positions[i, j]>ub
                lb_flag = positions[i, j]>lb

                if ub_flag:
                    positions[i,j] = ub
                if lb_flag:
                    positions[i,j] = lb

            rbf_regressor = svm.SVR(
                                    kernel = 'rbf',
                                    C = positions[i][0],
                                    gamma = positions[i][1]
                                    ).fit(X_train, y_train)
            cv_accuracies = cross_val_score(
                                            rbf_regressor,
                                            X_test,
                                            y_test
                                            ,cv =3,scoring = 'neg_mean_squared_error'
                                            )

            accuracies = cv_accuracies.mean()
            fitness_value = (1 - accuracies)*100

            if fitness_value < alpha_score:
                alpha_score = fitness_value
                alpha_position = positions[i]
            if fitness_value > alpha_score and fitness_value < beta_score:
                beta_score = fitness_value
                beta_position = positions[i]
            if fitness_value > alpha_score and fitness_value > beta_score and fitness_value < delta_score:
                delta_score = fitness_value
                delta_position = positions[i]

        a = 2-t*(2/T)

        for i in range(0,(positions.shape[0])):
            for j in range(0,(positions.shape[1])):
                r1=np.random(1)
                r2=np.random(1)
                A1=2*a*r1-A
                C1 = 0.5 + (0.5*math.exp(-j/500)) + (1.4*(math.sin(j)/30))

                D_alpha=abs(C1*alpha_position[j]-positions[i,j])
                X1=alpha_position[j]-A1*D_alpha

                r1=np.random(1)
                r2=np.random(1)

                A2=2*a*r1-a
                #C2=2*r2
                C2  = 1 + (1.4*(1 - math.exp(-j/500)) ) + (1.4*(math.sin(j)/30))

                D_beta=abs(C2*beta_position[j]-positions[i,j])
                X2=beta_position[j]-A2*D_beta
                r1=np.random(1)
                r2=np.random(1)

                A3=2*a*r1-a
                #C3=2*r2
                C3=(1/(1+ math.exp(-0.0001*j/T) )) + ((0.5 - 2.5) * ((j/T) **2))

                D_delta=abs(C3*delta_position[j]-positions[i,j])
                X3=delta_position[j]-A3*D_delta

                positions[i,j]=(X1+X2+X3)/3

        t = t + 1
        iterations.append(t)
        accuracy.append((100-Alpha_score)/100)
        print(f'----------------Count of iterations----------------{t}')
        print(positions)
        print(f'C and gamma: {alpha_position}')
        print(f'accuracy: {(100-alpha_score)/100}')

    best_C=alpha_position[0]
    best_gamma=alpha_position[1]

    return best_C, best_gamma, iterations,accuracy

In [ ]:
def plot(iterations, accuracy):
    plt.plot(iterations,accuracy)
    plt.xlabel('Count of iterations',size = 20)
    plt.ylabel('Accuracy',size = 20)
    plt.title('Sanitized GWO-SVM Parameter Optimization (SGWO_SVM)')
    plt.show()